In [2]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI

import os
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file


True

In [3]:
api_key = os.getenv("OPENAI_API_KEY")

In [4]:
llm = ChatOpenAI(model_name="gpt-4", openai_api_key=api_key, temperature=0)

In [5]:
### Define a simple add function as a tool

@tool
def add_numbers(a: int, b: int) -> int:
    """
    Add two numbers and return the result.

    Args:
        a (int): The first number.
        b (int): The second number.

    Returns:
        int: The sum of the two numbers.
    """
    return a + b

In [6]:
tools = [add_numbers]

llm_with_tools = llm.bind_tools(tools)

In [8]:
@tool
def subtract_numbers(a: int, b: int) -> int:
    """
    Subtract two numbers and return the result.

    Args:
        a (int): The first number.
        b (int): The second number.

    Returns:
        int: The difference of the two numbers.
    """
    return a - b

@tool
def multiply_numbers(a: int, b: int) -> int:
    """
    Multiply two numbers and return the result.

    Args:
        a (int): The first number.
        b (int): The second number.

    Returns:
        int: The product of the two numbers.
    """
    return a * b

In [29]:
tool_map = {
    "add_numbers": add_numbers,
    "subtract_numbers": subtract_numbers,
    "multiply_numbers": multiply_numbers
}

In [30]:
### test the tools with llm_with_tools

input = {
    "a": 1,
    "b": 2
}

In [ ]:
tool_map["su"].invoke(input)

KeyError: 'subtract'

In [32]:
## add new tool in llm
tools = [add_numbers, subtract_numbers, multiply_numbers]
llm_with_tools = llm.bind_tools(tools)

In [33]:
llm_with_tools

RunnableBinding(bound=ChatOpenAI(profile={'max_input_tokens': 8192, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': False, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x00000242A0651A90>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000242A1BB9FD0>, root_client=<openai.OpenAI object at 0x00000242A0651190>, root_async_client=<openai.AsyncOpenAI object at 0x00000242A1BB9AD0>, model_name='gpt-4', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True), kwargs={'tools': [{'type': 'function', 'function': {'name': 'add_numbers', 'description': 'Add two numbers and return t

In [34]:
query = "What is 10 minus 4?"

chat_history = [HumanMessage(content=query)]

In [35]:
response1 = llm_with_tools.invoke(chat_history)


In [36]:
chat_history.append(response1)

In [37]:
chat_history

[HumanMessage(content='What is 10 minus 4?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 217, 'total_tokens': 239, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-Cl8FaOQ36rRENTxm7DFcI0U0Nur2w', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b0702-7a87-7b52-8e63-8c1434e35174-0', tool_calls=[{'name': 'subtract_numbers', 'args': {'a': 10, 'b': 4}, 'id': 'call_WTiKQWHgJp7IMDnUXLJBrI5P', 'type': 'tool_call'}], usage_metadata={'input_tokens': 217, 'output_tokens': 22, 'total_tokens': 239, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_de

In [38]:
tool_calls_1 = response1.tool_calls
tool_calls_1

[{'name': 'subtract_numbers',
  'args': {'a': 10, 'b': 4},
  'id': 'call_WTiKQWHgJp7IMDnUXLJBrI5P',
  'type': 'tool_call'}]

In [39]:
too_call_name = tool_calls_1[0]["name"]
too_call_args = tool_calls_1[0]["args"]
too_call_id = tool_calls_1[0]["id"]

In [40]:
tool_response = tool_map[too_call_name].invoke(too_call_args)
tool_response

6

In [41]:
tool_message = ToolMessage(content=tool_response, tool_call_id=too_call_id)
tool_message

ToolMessage(content='6', tool_call_id='call_WTiKQWHgJp7IMDnUXLJBrI5P')

In [42]:
chat_history.append(tool_message)

In [43]:
chat_history

[HumanMessage(content='What is 10 minus 4?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 217, 'total_tokens': 239, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-Cl8FaOQ36rRENTxm7DFcI0U0Nur2w', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b0702-7a87-7b52-8e63-8c1434e35174-0', tool_calls=[{'name': 'subtract_numbers', 'args': {'a': 10, 'b': 4}, 'id': 'call_WTiKQWHgJp7IMDnUXLJBrI5P', 'type': 'tool_call'}], usage_metadata={'input_tokens': 217, 'output_tokens': 22, 'total_tokens': 239, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_de

In [44]:
answer = llm_with_tools.invoke(chat_history)

In [46]:
answer.content

'10 minus 4 is 6.'

In [53]:
### Building an agent

class ToolCallingAgent:
    def __init__(self):
        self.llm_with_tools = llm.bind_tools(tools)
        self.tool_map = tool_map

    def run(self, query: str) -> str:
        ## step 1: Initiate user message
        chat_history = [HumanMessage(content=query)]

        ## step 2: Get LLM response with tool calls
        response = self.llm_with_tools.invoke(chat_history)
        if not response.tool_calls:
            return response.content
        
        ## step 3: Process each tool call
        tool_calls = response.tool_calls[0]
        tool_name = tool_calls["name"]
        tool_args = tool_calls["args"]
        tool_id = tool_calls["id"]

        ## Step 4: call tool manually
        tool_result = self.tool_map[tool_name].invoke(tool_args)

        ## Step 5: Create tool message and append to chat history
        tool_message = ToolMessage(content=str(tool_result), tool_call_id=tool_id)
        chat_history.extend([response,tool_message])

        ## Step 6: Get final answer from LLM
        final_answer = self.llm_with_tools.invoke(chat_history)
        return final_answer.content

In [54]:
my_agent = ToolCallingAgent()
my_agent.run("What is 15 multiplied by 3?")

'15 multiplied by 3 is 45.'